In [1]:
from collections import namedtuple
import os
import tqdm
import tensorflow as tf
os.chdir("../")

In [2]:
from collections import namedtuple
from classifier_app.constants import *
from classifier_app.utils import read_yaml,create_directories
import os
import urllib.request as request
from zipfile import ZipFile

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size:int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksconfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    early_stopping_checkpoint_filepath: Path


In [4]:
from classifier_app.constants import *

class ConfigurationManager:


    def __init__(self, 
                    config_file_path = CONFIG_FILE_PATH,
                    params_filepath = PARAMS_FILE_PATH):
                    
                    self.config = read_yaml(config_file_path)
                    self.params = read_yaml(params_filepath)
                    
                    create_directories([Path(self.config.artifacts_root)])

    def get_training_config(self)-> TrainingConfig:
        config_training = self.config.training
        config_model = self.config.prepare_Base_Model
        training_data_path =os.path.join(self.config.data_ingestion.root_dir,"PetImages")

        params = self.params        
        create_directories([Path(config_training.root_dir)])

        trainingConfig = TrainingConfig(
                            root_dir=Path(config_training.root_dir),
                            trained_model_path=Path(config_training.trained_model_path),
                            updated_base_model_path = Path(config_model.updated_base_model_path),
                            training_data = Path(training_data_path),
                            params_epochs = self.params.EPOCHS,
                            params_batch_size= self.params.BATCH_SIZE,
                            params_is_augmentation= self.params.AUGMENTATION,
                            params_image_size= self.params.IMAGE_SIZE
                                )
                                
        return trainingConfig
    
    def get_prepareCallbackConfig(self)-> PrepareCallbacksconfig:
        config = self.config.prepare_callbacks
        params = self.params


        # checkpoint_model_dir, _  =  os.path.split(config.checkpoint_model_filepath)
        checkpoint_model_dir = os.path.dirname(config.checkpoint_model_filepath)
        early_stopping_checkpoint_dir = os.path.dirname(config.early_stopping_checkpoint_filepath)

        
        create_directories([Path(config.root_dir), Path(config.tensorboard_root_log_dir),
                            Path(checkpoint_model_dir),
                            Path(early_stopping_checkpoint_dir)])
        

        prepareCallbacksconfig = PrepareCallbacksconfig(
                                root_dir=Path(config.root_dir),
                                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
                                early_stopping_checkpoint_filepath= Path(config.early_stopping_checkpoint_filepath)
                                
                                )
                                
        return prepareCallbacksconfig    

In [5]:
import tensorflow as tf
import time

class PrepareCallback:

    def __init__(self, config: PrepareCallbacksconfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
                                        self.config.tensorboard_root_log_dir,
                                        f"tb_logs_at_{timestamp}"
        )
        print(tb_running_log_dir)                                
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only= True
        )

    @property
    def _create_earlyStopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor= "val_loss",
            min_delta= 0.001,
            patience=8,
            verbose = 0,
            mode = "auto",
            restore_best_weights=True            
        )


    def get_tb_ckpt_estopping_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_earlyStopping_callbacks
        ]



In [6]:
from statistics import mode
import tensorflow as tf
import time

class Training:

    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(filepath= self.config.updated_base_model_path)
        

    def train_Valid_generator(self):
        
        datagenerator_kargs = dict(
                                rescale = 1/255.0

                                )

        dataflow_kwargs = dict(
                            target_size = self.config.params_image_size[:-1],
                            batch_size = self.config.params_batch_size,
                            interpolation = "bilinear"
                            )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kargs)

        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                    rotation_range= 20,
                                    zoom_range= 0.20,
                                    width_shift_range= 0.20,
                                    height_shift_range= 0.20,
                                    shear_range= 0.20,
                                    horizontal_flip= True,
                                    vertical_flip= True,
                                    **datagenerator_kargs
            )
        else:
            train_datagen = valid_datagen

        self.train_generator = train_datagen.flow_from_directory(
                            directory= self.config.training_data,
                            subset= "training",
                            shuffle= True,
                            **dataflow_kwargs            
                            )

        self.valid_generator = valid_datagen.flow_from_directory(
                                directory= self.config.training_data,
                                shuffle= False,
                                subset = "validation",
                                **dataflow_kwargs
                                )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(filepath= path)

    def train(self, callbackList: list):
        steps_per_epoch = self.train_generator.samples//self.train_generator.batch_size
        validation_steps = self.valid_generator.samples// self.valid_generator.batch_size

        self.model.fit(
                    self.train_generator,
                    validation_data = self.valid_generator,
                    steps_per_epoch = steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs= self.config.params_epochs,
                    callbacks = callbackList
        )

        self.save_model(path = self.config.trained_model_path, model = self.model)   

    

In [7]:
try :

    config = ConfigurationManager()
    prepareCallbacks = PrepareCallback(config= config.get_prepareCallbackConfig())
    callbacklist = prepareCallbacks.get_tb_ckpt_estopping_callbacks()


    training = Training(config= config.get_training_config())
    training.get_base_model()
    training.train_Valid_generator()
    training.train(callbackList= callbacklist)
    

except Exception as e:
    raise e

artifacts\prepare_callbacks\tensorboard_log_dir\tb_logs_at_2022-10-29-01-42-05
Found 24998 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


UnknownError: Graph execution error:

Detected at node 'model_2/block2_conv2/Relu' defined at (most recent call last):
    File "d:\VS_projects\CNN_Classifier_App\env\lib\runpy.py", line 192, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\VS_projects\CNN_Classifier_App\env\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "d:\VS_projects\CNN_Classifier_App\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\VS_projects\CNN_Classifier_App\env\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "d:\VS_projects\CNN_Classifier_App\env\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\VS_projects\CNN_Classifier_App\env\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\VS_projects\CNN_Classifier_A

In [ ]:
import time

time.strftime("%Y-%m-%d-%H-%M-%S")

'2022-10-28-17-12-27'